In [4]:
import cv2
import numpy as np
import os

def load_images_from_folder(folder):
    images = []  # List to store loaded grayscale images
    labels = []  # List to store corresponding integers for each image
    for label, person_folder in enumerate(os.listdir(folder)):  # Iterate through each subdirectory
        person_path = os.path.join(folder, person_folder)
        if os.path.isdir(person_path):  # Check if person_path is indeed a directory
            for filename in os.listdir(person_path):
                img_path = os.path.join(person_path, filename)
                img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:  # If image exists
                    images.append(img)
                    labels.append(label)  # Unique label for each person
    return images, labels  # Return lists of images and labels

# Load images and labels
images, labels = load_images_from_folder('dataset')  # Ensure 'dataset' contains subfolders for each person

# Initialize and train recognizer
face_recognizer = cv2.face.LBPHFaceRecognizer_create()
face_recognizer.train(images, np.array(labels))

# Load Haar cascade for face detection
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Capture video from webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)

    for (x, y, w, h) in faces:
        roi_gray = gray[y:y+h, x:x+w]  # Region of interest for prediction
        label, confidence = face_recognizer.predict(roi_gray)

        # Set a threshold for confidence
        if confidence < 100:  # Adjust this threshold based on testing
            name = f"Kidist {label}"  # Replace with actual names if available
        else:
            name = "Unknown"  # Correctly classify as unknown

        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Draw rectangle around face
        cv2.putText(frame, name, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)  # Put text on frame

    cv2.imshow('Face Recognition', frame)  # Display the resulting frame

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Break the loop on 'q' key press
        break

# Release the capture and close windows
cap.release()
cv2.destroyAllWindows()

In [21]:
#Basic concepts to understand in faceRecognition source code
#import os-to create,remove and list directories and paths
#import numpy as kid(numerical python)-for arrays,matrices and numerical computing
#import cv2-to read,write and manipulate images and videos
#